Код к соревнованию https://www.kaggle.com/competitions/nlp-getting-started/  
Задача - научить модель распознавать твиты, в которых есть информация о природных катаклизмах, а в каких нет. Датасет состоит из 10000 твитов, размеченных вручную.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from string import punctuation
punctuation += "«»—…“”"

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model, model_selection, preprocessing

Импортируем датасеты чтобы обучать модель и проверять качество:

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
print(train_df.shape, test_df.shape)

(7613, 5) (3263, 4)


Посмотрим на примеры:

In [3]:
# нейтральный твит
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [4]:
# твит про катаклизм
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

Проведём предобработку текстов: уберём пунктуацию и приведём слова к первоначальной форме.

In [5]:
import nltk
from nltk.stem import WordNetLemmatizer # класс лемматизатор
from nltk.corpus import wordnet # WordNet — это лексическая база данных английского языка
from nltk.tokenize import word_tokenize # разделяет предложение по пробелам, на выходе список

lemmatizer = WordNetLemmatizer()

In [6]:
# nltk.download('averaged_perceptron_tagger')

In [7]:
# вспомогательная функция для лемматизатора
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [9]:
def preprocess(sentence):
    return ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)).lower() for w in nltk.word_tokenize(sentence)\
            if w not in punctuation])

In [12]:
# сделать векторы с предобработанными твитами
preprocessed_train = [preprocess(sentence) for sentence in train_df["text"]]
preprocessed_test = [preprocess(sentence) for sentence in test_df["text"]]

Векторизуем тексты:

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [15]:
train_vectors = tfidf_vectorizer.fit_transform(pd.Series(preprocessed_train))
                                              
## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = tfidf_vectorizer.transform(pd.Series(preprocessed_test))

Инициализируем классификатор:

In [16]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

Проверяем качество полученной модели:

In [17]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.6401742 , 0.61984733, 0.67342799])

Я пробовала ещё два классификатора, но они показали результаты хуже.

In [19]:
# from sklearn.ensemble import BaggingClassifier
# clf = BaggingClassifier()

In [20]:
# scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
# scores

In [21]:
# from sklearn.ensemble import GradientBoostingClassifier
# clf = GradientBoostingClassifier()

In [22]:
# scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
# scores

Подготавливаем файл для соревнования:

In [23]:
clf.fit(train_vectors, train_df["target"])
# X{ndarray, sparse matrix} of shape (n_samples, n_features)
# Training data.

# yndarray of shape (n_samples,)
# Target values.

RidgeClassifier()

In [24]:
sample_submission = pd.read_csv("sample_submission.csv")

In [25]:
sample_submission["target"] = clf.predict(test_vectors)

In [26]:
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [27]:
sample_submission.to_csv("submission.csv", index=False)